# **Principal Component Analysis**

## **Import useful Python packages**

In [0]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## **Check everything is ok**

In [0]:
spark

In [0]:
sc._conf.getAll()

# **Data Acquisition**

## **MNIST Dataset**

[MNIST](http://yann.lecun.com/exdb/mnist/) ("Modified National Institute of Standards and Technology") is the de facto "Hello World" dataset of computer vision. Since its release in 1999, this classic dataset of handwritten digit images has served as the basis for benchmarking classification algorithms.

### Download the dataset to the local driver node's ```/tmp``` folder using ```wget```

In [0]:
%sh wget -P /tmp https://github.com/gtolomei/big-data-computing/raw/master/datasets/mnist-train.csv.bz2

In [0]:
%fs ls file:/tmp/

### Move the file from local driver node's file system to DBFS

In [0]:
dbutils.fs.mv("file:/tmp/mnist-train.csv.bz2", "dbfs:/bdc-2020-21/datasets/mnist-train.csv.bz2")

In [0]:
%fs ls /bdc-2020-21/datasets/

### Read dataset file into a Spark Dataframe

In [0]:
mnist_df = spark.read.load("dbfs:/bdc-2020-21/datasets/mnist-train.csv.bz2", 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

### Check the shape of the loaded dataset, i.e., number of rows and columns

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(mnist_df.count(), len(mnist_df.columns)))

### Print out the schema of the loaded dataset

In [0]:
mnist_df.printSchema()

### Dataset Shape and Schema

The dataset has 785 columns. The first column, called `label`, is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated 28-by-28 pixels image, i.e., a 784-dimensional vector.

Each pixel column in the dataset has a name like <code>pixel**k**</code>, where **`k`** is an integer between 0 and 783, inclusive. 
To locate this pixel on the image, suppose that we have decomposed **`k`** as **`k`** = `i * 28 + j`, where `i` and `j` are integers between 0 and 27, inclusive. Then <code>pixel**k**</code> is located on row `i` and column `j` of a 28 x 28 matrix, (indexing by zero).

For example, <code>pixel**31**</code> indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

### Display the first 5 rows of the dataset

In [0]:
mnist_df.show(5)

# **Data Preprocessing**

### Assembling Features into a single column

We make use of the `VectorAssembler` transformer, which combines a given list of columns into a single vector column.

In [0]:
columns = ['pixel{:d}'.format(k) for k in range(784)]

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=columns, 
                            outputCol="features")

In [0]:
mnist_df = assembler.transform(mnist_df)
mnist_df.select("features", "label").show(truncate=False)

### Eventually, retain only 2 columns: assembled features and label

In [0]:
mnist_df = mnist_df.select("features", "label")

# **Principal Component Analysis (PCA)**

In [0]:
mnist_df.show(5)

## **Standardize features**

As we know, PCA is highly sensitive to feature scale. Let's standardize each feature so that each feature value _x_ is transformed into _z_, as follows:
$$
  z = \frac{x-\mu}{\sigma}
$$
where _mu_ is the sample mean of the feature, and _sigma_ is the unbiased sample standard deviation (computed from all the observations).
In such a way, each $z$ will have 0-mean and 1-standard-deviation.

In [0]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", 
                        outputCol="std_features",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(mnist_df)

# Normalize each feature to have unit standard deviation.
mnist_df = scalerModel.transform(mnist_df)

In [0]:
mnist_df.show(5, truncate=False)

## **Run PCA**

In [0]:
K = 10 # number of principal components to extract

In [0]:
from pyspark.ml.feature import PCA

pca_model = PCA(k=K, inputCol="std_features", outputCol="pca_features")
pca_features = pca_model.fit(mnist_df)
pca_mnist_df = pca_features.transform(mnist_df).cache()

In [0]:
pca_mnist_df.show(5)

## **Plotting**

### Show the proportion of the data variance carried by each principal component (eigenvector)

In [0]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
_ = sns.barplot(x=[i for i in range(K)], 
                y=pca_features.explainedVariance.values, # `explainedVariance` returns the distribution of variance across eigenvectors, i.e., lambda_i/sum lambda_i
                ax=ax)
_ = ax.set_xlabel("Eigenvalues", labelpad=16, fontsize=16)
_ = ax.set_ylabel("Proportion of Variance", fontsize=16)
_ = ax.set_xticklabels(["lambda_{:d}".format(i) for i in range(K)], rotation=45)

### Transform the PySpark DataFrame into Pandas' DataFrame

In [0]:
pca_mnist_pdf = pca_mnist_df.toPandas()

In [0]:
pca_mnist_pdf.head()

### Plot data projected on the 2 principal components

In [0]:
fig, ax = plt.subplots(1,1,figsize=(12,8))
_ = plt.scatter(
    pca_mnist_pdf.pca_features.map(lambda x: x[0]), 
    pca_mnist_pdf.pca_features.map(lambda x: x[1]),
    c=pca_mnist_pdf.label, 
    edgecolor='none', 
    alpha=0.8,
    cmap=plt.cm.get_cmap('tab10', 10),
    axes=ax
    )
_ = ax.set_xlabel('PCA 1', labelpad=20, fontsize = 16)
_ = ax.set_ylabel('PCA 2', fontsize = 16)
plt.colorbar();

# **K-means Clustering**

### Function used for running K-means

In [0]:
RANDOM_SEED = 42

def k_means(dataset, 
            n_clusters, 
            distance_measure="euclidean", 
            max_iter=20, 
            features_col="features", 
            prediction_col="cluster", 
            random_seed=RANDOM_SEED):
  
  from pyspark.ml.clustering import KMeans

  print("""Training K-means clustering using the following parameters: 
  - K (n. of clusters) = {:d}
  - max_iter (max n. of iterations) = {:d}
  - distance measure = {:s}
  - random seed = {:d}
  """.format(n_clusters, max_iter, distance_measure, random_seed))
  # Train a K-means model
  kmeans = KMeans(featuresCol=features_col, 
                   predictionCol=prediction_col, 
                   k=n_clusters, 
                   initMode="k-means||", 
                   initSteps=5, 
                   tol=0.000001, 
                   maxIter=max_iter, 
                   seed=random_seed, 
                   distanceMeasure=distance_measure)
  model = kmeans.fit(dataset)

  # Make clusters
  clusters_df = model.transform(dataset).cache()

  return model, clusters_df

### Function used to evaluate obtained clusters

In [0]:
def evaluate_k_means(clusters, 
                     metric_name="silhouette", 
                     distance_measure="squaredEuclidean", # cosine
                     prediction_col="cluster"
                     ):
  
  from pyspark.ml.evaluation import ClusteringEvaluator
  
  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure, 
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)

### Run K-means by calling the function above

In [0]:
model, clusters_df = k_means(pca_mnist_df, 
                             10,
                             distance_measure="euclidean",
                             max_iter=1000, 
                             features_col="pca_features"
                             )

In [0]:
evaluate_k_means(clusters_df)

In [0]:
clusters_df.show(5)

In [0]:
clusters_df.groupBy("cluster").count().sort("cluster").show()

In [0]:
# Get unique values in the grouping column
clusters = sorted([x[0] for x in clusters_df.select("cluster").distinct().collect()])
print("Cluster IDs: [{:s}]".format(", ".join([str(c) for c in clusters])))

# Create a filtered DataFrame for each group in a list comprehension
cluster_list = [clusters_df.where(clusters_df.cluster == x) for x in clusters]

# Show the results
for x_id, x in enumerate(cluster_list):
  print("Showing the first 10 records of cluster ID #{:d}".format(x_id))
  x.select(["cluster", "label"]).show(10, truncate=False)

## **Compute Cluster Purity**

In [0]:
def purity(cluster_i):
  # get total count of items clustered in cluster_i
  n_i = cluster_i.count()
  # group by labels
  count_labels = cluster_i.groupBy("label").count()
  # find the maximum count
  max_i = count_labels.agg(max("count")).first()[0]
  # find the label associated with the maximum count
  mode = count_labels.join(
    count_labels.agg(max("count").alias("max_")), col("count") == col("max_")).limit(1).select("label")
  
  # finally, return the most frequent label (of the current cluster) along with its purity
  return mode.first()[0], max_i/n_i

In [0]:
purities = []
for x_id, x in enumerate(cluster_list):
  print("The most frequent label of cluster id #{:d} is {:d} [purity = {:.3f}]".format(x_id, purity(x)[0], purity(x)[1]))
  purities.append(purity(x)[1])

print("The overall average purity is: {:.3f}".format(np.mean(purities)))

## **Suggestion: Try Computing Precision, Recall, F1, and Adjusted Rand Index**

You can try computing additional external metrics as the ones indicated above. A good reference for doing this can be found at the following [link](https://towardsdatascience.com/evaluating-clustering-results-f13552ee7603)